# Introduction to the GPT API

In this notebook, we'll explore the basics of using the OpenAI GPT API.

In [ ]:
# Run this cell to setup the OpenAI client (please be patient, during initial setup)
exec('''import importlib.util\nimport subprocess\nimport sys\nimport os\nfrom getpass import getpass\n_client = None\ndef check_openai_installed():\n    if importlib.util.find_spec('openai') is None:\n        print('OpenAI package not found. Installing...')\n        subprocess.check_call([sys.executable, '-m', 'pip', 'install',\n            'openai'])\n        print('OpenAI package installed successfully.')\n    else:\n        print('OpenAI package is already installed.')\ndef import_openai():\n    check_openai_installed()\n    global OpenAI\n    from openai import OpenAI\ndef validate_api_key(api_key):\n    try:\n        client = OpenAI(api_key=api_key)\n        client.models.list()\n        return True\n    except Exception as e:\n        print(f'Error validating API key: {str(e)}')\n        return False\ndef get_api_key():\n    while True:\n        api_key = os.getenv('OPENAI_API_KEY')\n        if not api_key:\n            print('OPENAI_API_KEY not found in environment variables.')\n            api_key = getpass('Please enter your OpenAI API key: ')\n        if validate_api_key(api_key):\n            os.environ['OPENAI_API_KEY'] = api_key\n            return api_key\n        else:\n            print('Invalid API key. Please try again.')\n            os.environ.pop('OPENAI_API_KEY', None)\ndef setup_openai_client():\n    global _client\n    import_openai()\n    api_key = get_api_key()\n    _client = OpenAI(api_key=api_key)\n    print(f"API Key set: {'Yes' if _client.api_key else 'No'}")\n    print(\n        f"API Key (first 5 chars): {_client.api_key[:5] if _client.api_key else 'None'}"\n        )\n    return _client\ndef get_completion(prompt, model='gpt-3.5-turbo'):\n    global _client\n    if _client is None:\n        _client = setup_openai_client()\n    messages = [{'role': 'user', 'content': prompt}]\n    response = _client.chat.completions.create(model=model, messages=\n        messages, temperature=0)\n    return response.choices[0].message.content''')
client = setup_openai_client()

## 1. Basic Prompt Engineering
Let's start with a simple example of how changing the prompt can affect the output. Adding context to the prompt can improve the accuracy and relevance of the response.

In [ ]:
basic_prompt = "What is the capital of France?"
print("Basic prompt result:")
print(get_completion(basic_prompt))

improved_prompt = "As a knowledgeable geography expert, please tell me the capital of France and provide a brief description of its significance."
print("\nImproved prompt result:")
print(get_completion(improved_prompt))

## 2. Few-Shot Learning
Now, let's use few-shot learning to guide the model's behavior. This is a technique where the model is provided with a few examples of the desired output before being asked to generate its own output.

In [ ]:
few_shot_prompt = """
Classify the sentiment of the following reviews:

Review: "This movie was amazing!"
Sentiment: Positive

Review: "I hated every minute of it."
Sentiment: Negative

Review: "It was okay, I guess."
Sentiment: Neutral

Review: "The service at this restaurant is terrible."
Sentiment:
"""

print(get_completion(few_shot_prompt))

## 3. Chain-of-Thought Prompting
By providing a step-by-step breakdown of the problem, we can guide the model to a correct answer. This helps with complex reasoning tasks like word problems where we can help reduce ambiguity by clarifying the steps.

In [ ]:
cot_prompt = """
Solve the following word problem step by step:

Problem: If a train travels 120 km in 2 hours, how far will it travel in 5 hours assuming it maintains the same speed?

Step 1: Calculate the speed of the train
Step 2: Use the speed to determine the distance traveled in 5 hours

Please show your work for each step.
"""

print(get_completion(cot_prompt))

## 4. System Prompts
Now, let's use a system prompt to set the behavior of the AI assistant. System prompts are used to set the behavior of the AI assistant in a more programmatic way by offering parameters such as:

- **Model**: Specifies which version of the language model to use, like `gpt-3.5-turbo` or `gpt-4`. Different models can provide varying levels of comprehension and creativity.
- **Temperature**: Controls the randomness or creativity of the output. A lower temperature (e.g., 0) makes the output more deterministic and focused, while a higher temperature (e.g., 0.7) allows for more varied and creative responses.
- **Role**: Sets the assistant's persona or perspective, such as being a helpful assistant, a sarcastic commentator, or an expert in a specific field.

By adjusting these parameters, we can influence how the AI generates responses to better suit our requirements.

In [ ]:
def get_completion_with_system(system_prompt, user_prompt, model="gpt-3.5-turbo"):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

system_prompt = "You are a helpful assistant that always responds in rhyme."
user_prompt = "Explain how photosynthesis works."

print(get_completion_with_system(system_prompt, user_prompt))

## 5. Prompt Templates
Creating a reusable prompt template can save time and ensure consistency across different prompts. By defining a structured format for your prompts, you can easily modify key elements without having to rewrite the entire prompt each time. This approach not only enhances efficiency but also helps maintain a uniform tone and style across various interactions, making it easier to manage and scale your prompt engineering efforts.

In [ ]:
def product_description_template(product_name, key_features, target_audience):
    return f"""
    Create a compelling product description for {product_name}. 
    Key features: {', '.join(key_features)}
    Target audience: {target_audience}
    
    The description should be engaging, highlight the key features, and appeal to the target audience.
    """

template = product_description_template(
    "EcoBoost 3000 Electric Bike",
    ["500W motor", "50-mile range", "Lightweight aluminum frame"],
    "Urban commuters aged 25-45"
)

print(get_completion(template))

## 6. Retrieval-Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) is often regarded as a technique specifically designed for large language models (LLMs) because it enhances their ability to generate contextually relevant responses by integrating external knowledge sources. However, the essential idea behind RAGâ€”combining retrieval of information with generative capabilitiesâ€”is applicable beyond LLMs. It emphasizes the importance of grounding responses in factual data, which can improve the accuracy and relevance of outputs in various AI applications. For this example, we'll simulate a simple RAG system using a predefined knowledge base.

In [ ]:
knowledge_base = {
    "Python": "Python is a high-level, interpreted programming language known for its simplicity and readability.",
    "Machine Learning": "Machine Learning is a subset of AI that focuses on the development of algorithms that can learn from and make predictions or decisions based on data.",
    "Neural Networks": "Neural Networks are a series of algorithms that aim to recognize underlying relationships in a set of data through a process that mimics the way the human brain operates."
}

def rag_prompt(query):
    relevant_info = knowledge_base.get(query, "No specific information found in the knowledge base.")
    return f"""
    Using the following information from our knowledge base:
    '{relevant_info}'
    
    Please answer the user's question: '{query}'
    Provide additional context or examples if appropriate.
    """

user_query = "What is Python?"
print(get_completion(rag_prompt(user_query)))

## Conclusion
We've explored various prompting techniques, from basic prompt engineering to more advanced concepts like few-shot learning, chain-of-thought prompting, system prompts, prompt templates, and a simple example of RAG. These techniques demonstrate the flexibility and power of large language models, allowing us to achieve complex tasks without traditional model training.


## Exercises

Let's practice and expand on the prompting techniques we've learned with these exercises:

### Exercise 1: System Prompts and Role-Playing

Create a system prompt that instructs the AI to act as a historical figure. Then, write a user prompt asking a question related to that figure's expertise or time period. Use the `get_completion` function to generate a response.

In [ ]:
def historical_figure_chat(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = get_completion(messages)
    return response

# Example usage:
system_prompt = "You are Leonardo da Vinci, the Renaissance polymath. Respond as if you are living in the late 15th century, with knowledge of art, science, and engineering of that time."
user_prompt = "What are your thoughts on the future of human flight?"

response = historical_figure_chat(system_prompt, user_prompt)
print(response)

Now it's your turn! Choose a different historical figure and create a system prompt for them. Then, ask them a relevant question using the `historical_figure_chat` function.

In [ ]:
# Your code here
# Hint: system_prompt = "You are [historical figure]. ..."
#       user_prompt = "..."
#       response = historical_figure_chat(system_prompt, user_prompt)
#       print(response)

### Exercise 2: Chain-of-Thought Prompting for Problem Solving

Use chain-of-thought prompting to solve a multi-step problem. Create a prompt that guides the model through the problem-solving process step by step.

In [ ]:
def solve_problem_with_cot(problem_statement):
    prompt = f"""
    Solve the following problem step by step:
    
    Problem: {problem_statement}
    
    Please follow these steps:
    1. Identify the key information given in the problem.
    2. Determine what needs to be calculated.
    3. Choose the appropriate formula or method to solve the problem.
    4. Perform the calculations, showing your work.
    5. Check if the result makes sense in the context of the problem.
    6. State the final answer clearly.
    
    Show your work for each step and explain your reasoning.
    """
    return get_completion(prompt)

# Example usage:
problem = "A car travels 150 miles in 2.5 hours. What is its average speed in miles per hour?"
solution = solve_problem_with_cot(problem)
print(solution)

Now, create your own multi-step problem and use the `solve_problem_with_cot` function to solve it.

In [ ]:
# Your code here
# Hint: your_problem = "..."
#       your_solution = solve_problem_with_cot(your_problem)
#       print(your_solution)

### Exercise 3: Few-Shot Learning for Text Classification

Create a few-shot learning prompt to classify text into sentiment categories: "Positive", "Negative", or "Neutral". Provide examples for each category, then test your prompt with new text samples.

In [ ]:
def sentiment_classifier(text_to_classify):
    prompt = f"""
    Classify the sentiment of the following texts as Positive, Negative, or Neutral.

    Text: "I love this product! It's amazing."
    Sentiment: Positive

    Text: "This movie was terrible. I want my money back."
    Sentiment: Negative

    Text: "The weather is cloudy today."
    Sentiment: Neutral

    Text: "I can't believe how great this restaurant is!"
    Sentiment: Positive

    Text: "I'm feeling quite indifferent about the whole situation."
    Sentiment: Neutral

    Text: "{text_to_classify}"
    Sentiment:
    """
    return get_completion(prompt)

# Example usage:
text_sample = "I'm disappointed with the service I received."
classification = sentiment_classifier(text_sample)
print(f"Text: {text_sample}")
print(f"Classification: {classification}")

Now, test the sentiment classifier with at least three more text samples of your choice.

In [ ]:
# Your code here
# Hint: sample1 = "..."
#       print(f"Text: {sample1}")
#       print(f"Classification: {sentiment_classifier(sample1)}")
#       
#       # Repeat for sample2 and sample3